In [1]:
# Carrega as variáveis de embiente usadas pelo notebook. Estas variáveis devem ser criadas no arquivo .env, localizada no mesmo diretório do notebook
# As seguintes variáveis devem ser criadas:
# - export STR_CON=<string de conexão com o banco de dados relacional>
# - export RAW_DATA_PATH=<caminho onde os arquivos contendo o histórico de preço de cada ativo será armazenado>
# - export DATASET_PATH=<caminho onde os arquivos contendo o histórico de preço e os indicadores técnicos serão armazenados>
# - export MODELS_PATH=<caminnho onde os modelos treinados serão armazenados>
# - export TRAIN_DATASET=<caminho onde os datasets estandarizados e formatados para o treinamento serão armazenados>
%load_ext dotenv
%dotenv

# Garante que os scripts Python sejam recarregados em cada execução do notebook.
%load_ext autoreload
%autoreload 2

## Objetivo do Nobook 
<p>O objetivo deste notebook é extrair o histórico de preços de ações de banco de dados relacional, acrescentar os indicadores técnicos, rentabilidade e label de classificação e gravá-los como arquivos para que sejam usados no processo de treinamento do modelo de machine learning. O primeiro passo é obter os dados históricos de negociação de cada ativo gravá-los como arquivos individuais em disco. Estes arquivos devem conter as seguintes colunas:</p>
<ul>
<li>ticker - Código de negociação do ativo</li>
<li>dt_price - Data de negociação</li>
<li>open - Preço de abertura no dia</li>
<li>close - Preço de fechameto no dia</li>
<li>high - Preço máximo no dia</li>
<li>low - Preço mínimo no dia</li>
<li>volume - Volume financeiro negociado do ativo no dia</li>
</ul>
<p>É fundamental que os arquivos contenham exatamente estas colunas, pois a biblioteca TA-Lib (utilizada para calcularos indicadores técnicos) espera estes nomes como parâmetros de entrada para suas funções.</p>
<p><b>Exemplo</b>:</p>
<p>Lista de ativos recuperados do banco de dados: [PETR4, ITUB4, VALE3, ABEV3]</p>
<p>Arquivos gerados:</p>
<ul>
<li> PETR4.parquet </li>
<li> ITUB4.parquet </li>
<li> VALE3.parquet </li>
<li> ABEV3.parquet </li>
</ul>
<p>Uma vez gerados os arquivos <b>parquet</b> com os dados históricos de preço dos ativos, os indicadores técnicos serão calculados de acordo com as estratégias definias no arquivo de configuração <b>strategies.json</b> e a rentabilidade em X dias futuros calculada. Esta rentabilidade será usada para definir o label que será usado para classificar o ativo.</p>

### Imports

In [2]:
import os
import pandas as pd
import json
from data_preparation import PreProcess, PROFTABILITY_TYPE
from db_access import StockHistory, ExportToParquet, ExportToPickle

pd.options.display.max_columns = 200

### Geração dos arquivos apenas com o a série histórica de preços

In [3]:
# Obtem o historico de precos dos ativos 
str_conn = os.environ.get("STR_CON")
raw_file_path = os.environ.get("RAW_DATA_PATH")
data_file_path = os.environ.get("DATASET_PATH")
exporter = ExportToParquet()
dbo = StockHistory(str_conn)
df = dbo.select()
df["dt_price"] = df["dt_price"].astype('datetime64[ns]')
# Grava os arquivos com o histórico de preços em formato parquet em um diretório local
print(f"Gerando arquivos de preço")
for stock in df['ticker'].unique():
    df_aux = df.loc[df['ticker'] == stock]
    exporter.export(df_aux, raw_file_path, stock)
print(f"Arquivos de preço gerados com sucesso")

### Cálculo dos indicadores técnicos, e rentabilidade dos ativos
<p>O segundo passo da preparação dos datasets consiste em incluir os indicadores técnicos ao histórico de preço de cada ativo de acordo com as estratégias de negociação definidas no arquivo de configuração, calcular a rentabilidade do ativo X dias para frente e usar esta rentabilidade para definir o label de classificação do ativo (1 se o ativo se valorizou mais do que Y% X dias após a série de dados do registro do dataset ou 0 caso contrário).</p> 
<p>Uma estratégia de negociação nada mais é do que a combinação de indicadores técnicos que serão usados como features em modelos de machine learning. A ideia é realizar diversos experimentos com diferentes estratégias para encontrar combinações que ajudem melhorar a acurácia do modelo na classificação dos ativos com maior potencial de rentabilidade no período informado.</p>
<p>Para automatizar o processo, as estratégias de negociação são configuradas em um arquivo json com a seguinte estrutura:</p>
<code>
<pre>
    "&lt;nome da estrategia&gt;":{
        "description": "&lt;Descrição da estratégia&gt;",
        "historic_period": &lt;Número inteiro que irá definir o total dias usados pelo modelo. Por exemplo, o modelo receberá um histórico de preço de 
                            60 dias para classificar o ativo&gt;,
        "profit_period": &lt;Número inteiro que definirá o período usado para calcular a rentabilidade do ativo (será usado para definir seu label).
                          Por exemplo, o modelo reberá 60 dias de cotação para classificar o ativo, se este parâmetro for setado para 10, a rentablidade 
                          usada para definir o label será calclada como: 
                          preço 10 dias após a o último dia da série de entrada/preço do último dia da série de entrada - 1&gt;,
        "shift_days": &lt;Número inteiro que define quantos dias serão "pulados" durante a geração das series temporais de preços. Por exemplo, um modelo
                       poderá ser treinado com um histórico de preços de 30 dias, a primeira série histórica será do dia 01 até o dia 30 do mês, se
                       shift_days for igual a 5 dias, a segunda série será do dia 6 (cinco dias após o início da série anterior) até o dia 6 mais 30
                       dias&gt;,
        "profit": &lt;Rentabilidade usada para definir o label de classificação de cada registro enviado para o modelo durante o treinamento. Por exemplo,
                   Se o objetivo do modelo for identificar ativos que possuam rentabiliade superior a 5% em um período de 20 dias, o valor atribuído para
                   este parâmetro deve ser 0.05&gt;,
        "functions": { -- Lista de funções que fazem parte da estratégia de negociação
            "&lt;nome "fantasia" da função&gt;":{
                "function": "&lt;nome da função na biblioteca TA-Lib, será usado para chamar a função no código&gt;",
                "params":{ -- Lista de parâmetros da função (parâmetros relacionados ao preço do ativo não devem ser listados aqui)
                    "&lt;nome da parâmetro&gt;": &lt;valor&gt;
                }                
            }
        }
    }
</pre>
</code>
<p>Exemplo de configuração:</p>
<p>Neste exemplo, a estratégia considera 90 dias de histórico de preço do ativo para classificar ativos que tenham rentabilidade superior a 5% em um período de 22 dias e para fazer isso, utiliza além do histórico de cotações os indicadores: Média móvel exponencial de 7 e 21 dias e o OBV.</p>
<code>
<pre>
{
    "EMA_7_21_OBV":{
        "description": "Combinacao de 2 medias moveis (7 e 21 dias) e OBV ",
        "historic_period": 90,
        "profit_period": 22,
        "shift_days": 5,
        "profit": 0.05,
        "functions": {
            "EMA_7":{
                "function": "EMA",
                "params":{
                    "timeperiod": 7
                }
            },
            "EMA_21": {
                "function": "EMA",
                "params":{
                    "timeperiod": 21
                }            
            },
            "OBV":{
                "function": "OBV"                  
            }
        }
    }
}
</pre>
</code>
<p>Após calcular todos os indicadores técnicos, o dataset será formatado para que as séries históricas possam ser processadas pelo primeiro modelo, que é um Autoencoder com camadas LSTM. Camadas LSTM esperam como entrada tensores com 3 dimensões (batch, timesteps, features), onde:
<pre>
batch - representa o número de registros que serão enviados ao modelo em cada iteração do treinamento (batch size)
timesteps - quantidade de dias da série temporal que serão passadas para a camada LSTM (historic_period)
features - quantidade de colunas do dataset de entrada, por exemplo se a estratégia utilizar o preço de fechamento, médias móveis de 7 e 21 dias, o dataset de entrada terá 3 features.
</pre>
<p>Além da série histórica de preços de cada ativo, o dataset de treinamento conterá o código de negociação do ativo (ticker), datas de início e fim da série temporal contida em cada registro, rentabiliade X dias a frente e o label utilizado para treinar o modelo que irá classificar os ativos. As colunas ticker, datas de início e fim da série temporal e rentabilidade não serão usadas durante o treinamento dos modelos, elas existem apenas para facilitar a validação dos dados de entrada.</p>
<pre>
Exemplo:<br/>
    * historic_period = 20
    * profit_period = 10
    * stride = 5
    * profit = 0.05
    * Valores da estratégia = close, MME7, MME21
    
Cada registro do dataset de treinamento conterá as seguintes colunas:</br>
Ticker  Data Inicio     Data Fim         Serie Histórica                     Rentabilaide    Label
XXXX4   2022-07-04      2022-07-29       18.48   18.631468   18.727143       -0.021312       0
&nbsp;                                        18.48   18.573601   18.697403
&nbsp;                                        18.50   18.490201   18.655821
&nbsp;                                        18.69   18.552069   18.579502
&nbsp;                                        .         .            .
&nbsp;                                        .         .            .
&nbsp;                                        .         .            .
&nbsp;                                        19.03   18.661552   18.616820
XXXX4   2022-07-11      2022-08-05       18.48   18.631468   18.727143       0.0621312       1
&nbsp;                                        18.48   18.573601   18.697403
&nbsp;                                        18.50   18.490201   18.655821
&nbsp;                                        18.69   18.552069   18.579502
&nbsp;                                        .         .            .
&nbsp;                                        .         .            .
&nbsp;                                        .         .            .
&nbsp;                                        19.03   18.661552   18.616820
</pre>
<p>Ao final do processo, arquivos pickle serão gravados contendo os valores calculados e formatados. Cada estratégia irá gerar um diretório contendo os arquivos com os dados e existirá um arquivo por ativo processado.</p>

In [28]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pre_process = PreProcess()
parquet_exporter = ExportToParquet()
pickle_exporter = ExportToPickle()

raw_file_path = os.environ.get("RAW_DATA_PATH")
data_file_path = os.environ.get("DATASET_PATH")
model_base_path = os.environ.get("MODELS_PATH")
columns_to_pop = ['ticker', 'dt_price']

path_content = os.listdir(raw_file_path)
# Filtra os arquivos parquet do diretório
path_content = [file for file in path_content if file.endswith(".parquet")]

for file in path_content:
    print(f"Processando arquivo: {raw_file_path}/{file}")
    df_raw = pd.read_parquet(os.path.join(raw_file_path, file))
    df_raw.set_index("dt_price", drop=False, inplace=True)

    # Calcula os indicadores técnicos definidos nas estratégias para cada ativo
    for strategy_name, strategy, df_tech in pre_process.calculate_strategy("strategies.json", df_raw):
        window_size = strategy["historic_period"]
        stride = strategy["stride"]
        profit_period = strategy["profit_period"]
        profit = strategy["profit"]

        df_aux = pd.DataFrame()

        # Remove os valores nulos do DataFrame
        df_tech.dropna(inplace=True)

        for col in columns_to_pop:
            df_aux[col] = df_tech.pop(col)

        df_aux.reset_index(inplace=True, drop=True)

        # Cria um MinMaxScaler para fazer o scaling das features
        scaler = StandardScaler().fit(df_tech)
        # Salva o scaling para ser usado após o treinamento
        pickle_exporter.export(scaler, os.path.join(model_base_path, strategy_name), file.split('.')[0])

        # Gera o dataframe com os dados redimensionados
        transformed_data = scaler.transform(df_tech)

        df_tech = pd.concat([df_aux, pd.DataFrame(transformed_data, columns=scaler.get_feature_names_out())], axis=1)

        df_tech.set_index('dt_price', drop=False, inplace=True)

        # Formata o dataset para ser usado no modelo
        df_model = pre_process.format_dataset(df_raw, df_tech, window_size, stride, profit_period, profit, None)
 
        # Exporta o data set como um arquivo pickle (arquivos parquet não suportam arrays multidimensionais)
        parquet_exporter.export(df_model, os.path.join(data_file_path, strategy_name), file.split('.')[0])


Processando arquivo: data/raw/CPFE3.parquet
Processing strategy: Bolinger Bands 21 dias, MACD (12, 26, 9) e AD
Calculating BBANDS_14...
Calculating MACD...
Calculating AD...
Calculating NATR...
Calculating RSI...
Calculating WILLR...
Processando arquivo: data/raw/PMAM3.parquet
Processing strategy: Bolinger Bands 21 dias, MACD (12, 26, 9) e AD
Calculating BBANDS_14...
Calculating MACD...
Calculating AD...
Calculating NATR...
Calculating RSI...
Calculating WILLR...
Processando arquivo: data/raw/BEEF3.parquet
Processing strategy: Bolinger Bands 21 dias, MACD (12, 26, 9) e AD
Calculating BBANDS_14...
Calculating MACD...
Calculating AD...
Calculating NATR...
Calculating RSI...
Calculating WILLR...
Processando arquivo: data/raw/LOGG3.parquet
Processing strategy: Bolinger Bands 21 dias, MACD (12, 26, 9) e AD
Calculating BBANDS_14...
Calculating MACD...
Calculating AD...
Calculating NATR...
Calculating RSI...
Calculating WILLR...
Processando arquivo: data/raw/VIVT3.parquet
Processing strategy:

### Criação dos datasets usados no treinamento
<p>As últimas etapas da perparação de dados é a divisão do dataset em dois subconjuntos (treinamento e validação) e o redimensionamento dos valores, para que todos os números tenham a mesma ordem de grandeza (números entre 0 e 1). O processo de redimensionamento dos valores é feito por ativo, e os datasets resultantes são concatenados, gerando um dataset completo que será divido em 2 (treino e teste) e gravado como arquivo parquet para ser usado no treinamento do modelo.</p>

In [30]:
pre_process = PreProcess()
random_seed = 42
test_size = 0.15

pre_process.create_train_test_dataset("strategies.json", test_size, random_seed)

Processando arquivo CPFE3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo PMAM3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo BEEF3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo LOGG3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo VIVT3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo PLPL3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo KEPL3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo ROMI3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo HAPV3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo ARZZ3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo SOMA3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo PETR3.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo TRPL4.parquet na estrategia BBANBDS_21_MACD_12_26_9_AD
Processando arquivo AGRO3